# Phase 4: Modeling

**Project**: [Project Name]  
**Author**: Jose Marcel Lopez Pino  
**Date**: [YYYY-MM-DD]  
**Framework**: CRISP-DM + Lean  

---

## Objective

Build analytical models to answer the business questions.
Start with the simplest approach that provides value.

**Lean Filter**: Simplest model that answers the question first.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')
from config.config import DATA_PROCESSED, DATA_FINAL

## 4.1 Load Prepared Data

In [ ]:
# df = pd.read_csv(DATA_PROCESSED / 'dataset_clean.csv')

## 4.2 Approach Selection

| Approach | Complexity | Justification |
|----------|-----------|---------------|
| [Option A — simple] | Low | [Why it might suffice] |
| [Option B — complex] | High | [Why it might be needed] |

**Selected**: [Approach] — [Rationale]

## 4.3 Analysis / Model

In [ ]:
# Implement selected approach

## 4.4 Results

In [ ]:
# Present results with visualizations

---

**Previous**: [03_data_preparation.ipynb](./03_data_preparation.ipynb)  
**Next**: [05_evaluation.ipynb](./05_evaluation.ipynb)